In [ ]:
import os
import numpy as np
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import random

samples = []
labels = []
images_folder = "images/"
image_list = os.listdir(images_folder)
random.seed(10)
random.shuffle(image_list)

for image in image_list:
    samples.append(img_to_array(load_img(images_folder + image, target_size = (200, 200)))) # Change Target Size accordingly to the resolution.
#     print(image)
    if "normal" in image:
        labels.append((0))
    else:
        labels.append((1))
        
samples = np.array(samples)
labels = np.array(labels)
eval_x, eval_y = samples[-100:], labels[-100:] # 2000 images, 100 in evaluation.
print(samples.shape, labels.shape)

x_train, x_test, y_train, y_test = train_test_split(samples[:-100], labels[:-100], test_size = 0.2, random_state=31)

In [ ]:
%%time

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import History 


model = Sequential()

model.add(keras.layers.InputLayer(input_shape=(200, 200, 3)))

model.add(keras.layers.convolutional.Conv2D(filters = 16, kernel_size = (3, 3), strides=(1, 1), padding='valid', data_format="channels_last", activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=1, input_dim=50,activation='sigmoid'))

print(model.summary())
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
history = History()
model.fit(x_train, y_train, batch_size = 32, epochs = 50,  callbacks = [history], validation_data = (x_test, y_test), verbose = 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plot the training loss and accuracy
def plotResults(): 
    plt.figure()
    N = 50
    
    # TODO: plot the accuracy/loss variables over training time
    plt.plot(np.arange(0, N), history.history["accuracy"], label = "train_acc")
    plt.plot(np.arange(0, N), history.history["val_accuracy"], label = "val_acc")

    # make the graph understandable: 
    plt.title("Training Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()
    
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label = "train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label = "val_loss")
    
     # make the graph understandable: 
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.yscale('log')
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
model.save_weights('basic-model.h5')
plotResults()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

def evaluate_on_training_set(y_test, y_pred):
    
    # Calculate AUC
    print("AUC  is:  ",  roc_auc_score(y_test,  y_pred))
    
    # print out recall and precision
    print(classification_report(y_test, y_pred))
    
    # print out confusion matrix
    print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
    
    # # calculate points for ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    
    # Plot ROC curve
    plt.plot(fpr,  tpr,  label='ROC  curve  (area  =  %0.3f)'  %  roc_auc_score(y_test,  y_pred)) 
    plt.plot([0,  1],  [0,  1],  'k--') #  random  predictions  curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False  Positive  Rate  or  (1  -  Specifity)') 
    plt.ylabel('True  Positive  Rate  or  (Sensitivity)') 
    plt.title('Receiver  Operating  Characteristic')


In [ ]:
score = model.evaluate(eval_x, eval_y, verbose = 1)

In [ ]:
import tensorflow as tf
output = model.predict_classes(eval_x, verbose = 1)
pred_y = np.array(output).flatten()

In [ ]:
print(f"Loss: {score[0]}\nAccuracy: {score[1]}")
evaluate_on_training_set(eval_y, pred_y)